In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['EMT']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('epithelial mesenchymal transition', 13517),
 ('epithelial to mesenchymal', 140),
 ('epithelial mesenchymal transformation', 133),
 ('epithelial to mesenchymal transformation', 66),
 ('epithelial mesenchymal transdifferentiation', 33),
 ('epithelialmesenchymal transition', 21),
 ('endothelial to mesenchymal transformation', 17),
 ('epithelial to mesenchymal transdifferentiation', 12),
 ('endothelial mesenchymal transformation', 12),
 ('mesenchymal epithelial transition', 10),
 ('endocardial to mesenchymal transformation', 8),
 ('epithelial mensenchymal transition', 8),
 ('epithelial mesenchymal cells transition', 8),
 ('epitheliomesenchymal transition', 7),
 ('mesenchymal to epithelial transition', 7),
 ('extraneuronal monoamine transporter', 6),
 ('endometrial thickness', 6),
 ('eminence', 6),
 ('endothelial mesenchymal transdifferentiation', 5),
 ('epithelial mesenchymal transition process', 5),
 ('epithelial to mesenchymal phenotype', 5),
 ('epithelial tomesenchymal transition', 5

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'GO:GO:0060290': 'transdifferentiation',
 'GO:GO:0001837': 'epithelial to mesenchymal transition',
 'HGNC:10967': 'SLC22A3',
 'MESH:D010641': 'Phenotype',
 'MESH:D055032': 'Electron Microscope Tomography',
 'HP:HP:0025153': 'Transient',
 'MESH:D017131': 'Thrombectomy',
 'GO:GO:0060231': 'mesenchymal to epithelial transition',
 'HGNC:6179': 'ITK'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-31 02:59:35] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'eastern mediterranean transient': 'ungrounded',
  'em tomography': 'MESH:D055032',
  'eminence': 'ungrounded',
  'emt': 'ungrounded',
  'emt itk tsk': 'ungrounded',
  'endocardial mesenchymal transformation': 'ungrounded',
  'endocardial to mesenchymal transformation': 'ungrounded',
  'endometrial thickness': 'ungrounded',
  'endothelial mesenchymal transdifferentiation': 'ungrounded',
  'endothelial mesenchymal transformation': 'ungrounded',
  'endothelial to mesenchymal transformation': 'ungrounded',
  'endovascular mechanisms thrombectomy': 'ungrounded',
  'epithelial cells to mesenchymal cells': 'GO:GO:0001837',
  'epithelial mensenchymal transition': 'GO:GO:0001837',
  'epithelial mesenchimal transition': 'GO:GO:0001837',
  'epithelial mesenchymal cells transition': 'GO:GO:0001837',
  'epithelial mesenchymal like transition': 'GO:GO:0001837',
  'epithelial mesenchymal transdifferentiation': 'GO:GO:0001837',
  'epithelial mesenchymal transformation': 'GO:GO:0001837',
  'epitheli

In [11]:
grounding_map, names, pos_labels = [{'eastern mediterranean transient': 'ungrounded',
  'em tomography': 'MESH:D055032',
  'eminence': 'ungrounded',
  'emt': 'ungrounded',
  'emt itk tsk': 'ungrounded',
  'endocardial mesenchymal transformation': 'ungrounded',
  'endocardial to mesenchymal transformation': 'ungrounded',
  'endometrial thickness': 'ungrounded',
  'endothelial mesenchymal transdifferentiation': 'ungrounded',
  'endothelial mesenchymal transformation': 'ungrounded',
  'endothelial to mesenchymal transformation': 'ungrounded',
  'endovascular mechanisms thrombectomy': 'ungrounded',
  'epithelial cells to mesenchymal cells': 'GO:GO:0001837',
  'epithelial mensenchymal transition': 'GO:GO:0001837',
  'epithelial mesenchimal transition': 'GO:GO:0001837',
  'epithelial mesenchymal cells transition': 'GO:GO:0001837',
  'epithelial mesenchymal like transition': 'GO:GO:0001837',
  'epithelial mesenchymal transdifferentiation': 'GO:GO:0001837',
  'epithelial mesenchymal transformation': 'GO:GO:0001837',
  'epithelial mesenchymal transition': 'GO:GO:0001837',
  'epithelial mesenchymal transition or transformation': 'GO:GO:0001837',
  'epithelial mesenchymal transition process': 'GO:GO:0001837',
  'epithelial mesencymal transition': 'GO:GO:0001837',
  'epithelial myofibroblasts transdifferentiation': 'ungrounded',
  'epithelial to mesechymal transition': 'GO:GO:0001837',
  'epithelial to mesenchimal transition': 'GO:GO:0001837',
  'epithelial to mesenchymal': 'GO:GO:0001837',
  'epithelial to mesenchymal like transition': 'GO:GO:0001837',
  'epithelial to mesenchymal phenotype': 'GO:GO:0001837',
  'epithelial to mesenchymal transdifferentiation': 'GO:GO:0001837',
  'epithelial to mesenchymal transformation': 'GO:GO:0001837',
  'epithelial to mesencyhmal transition': 'GO:GO:0001837',
  'epithelial to myofibroblasts transition': 'GO:GO:0001837',
  'epithelial tomesenchymal transition': 'GO:GO:0001837',
  'epithelialmesenchymal transition': 'GO:GO:0001837',
  'epitheliomesenchymal transition': 'GO:GO:0001837',
  'expiratory muscle training': 'ungrounded',
  'extraneuronal monoamine transporter': 'HGNC:10967',
  'eye movement training': 'ungrounded',
  'mesenchymal epithelial transition': 'GO:GO:0001837',
  'mesenchymal to epithelial transition': 'GO:GO:0001837',
  'the epithelial mesenchymal': 'GO:GO:0001837'},
 {'MESH:D055032': 'Electron Microscope Tomography',
  'GO:GO:0001837': 'epithelial to mesenchymal transition',
  'HGNC:10967': 'SLC22A3'},
 ['GO:GO:0001837', 'HGNC:10967']]

In [12]:
excluded_longforms = ['emt']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {'HGNC:10967': ['SLC22A3', ['EMT', 'extraneuronal monoamine']]}

In [16]:
unambiguous_agent_texts = {'HGNC:10967': ['SLC22A3', ['extraneuronal monoamine transporter']]}

In [17]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [24]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 03:06:04] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 03:08:00] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9957503486264458 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [25]:
classifier.stats

{'label_distribution': {'GO:GO:0001837': 10335,
  'ungrounded': 56,
  'HGNC:10967': 15,
  'MESH:D055032': 2},
 'f1': {'mean': 0.99575, 'std': 0.000408},
 'precision': {'mean': 0.994819, 'std': 0.00072},
 'recall': {'mean': 0.996908, 'std': 0.000785},
 'HGNC:10967': {'f1': {'mean': 0.531429, 'std': 0.304175},
  'pr': {'mean': 0.466667, 'std': 0.339935},
  'rc': {'mean': 0.75, 'std': 0.387298}},
 'GO:GO:0001837': {'f1': {'mean': 0.996424, 'std': 0.000417},
  'pr': {'mean': 0.997678, 'std': 0.000987},
  'rc': {'mean': 0.995175, 'std': 0.000523}},
 'MESH:D055032': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.33254, 'std': 0.138022},
  'pr': {'mean': 0.287879, 'std': 0.136195},
  'rc': {'mean': 0.419365, 'std': 0.148041}}}

In [26]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [27]:
disamb.dump(model_name, results_path)

In [28]:
print(disamb.info())

Disambiguation model for EMT

Produces the disambiguations:
	Electron Microscope Tomography	MESH:D055032
	SLC22A3*	HGNC:10967
	epithelial to mesenchymal transition*	GO:GO:0001837

Class level metrics:
--------------------
Grounding                           	Count	F1     
epithelial to mesenchymal transition*	10335	0.99642
                          Ungrounded	   56	0.33254
                             SLC22A3*	   15	0.53143
      Electron Microscope Tomography	    2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99575
	Precision:	0.99482
	Recall:		0.99691

* Positive labels
See Docstring for explanation



In [29]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1